# Семинар 5    
## Собираем поисковик 

![](https://bilimfili.com/wp-content/uploads/2017/06/bir-urune-emek-vermek-o-urune-olan-deger-algimizi-degistirir-mi-bilimfilicom.jpg) 


Мы уже все знаем, для того чтобы сделать поисковик. Осталось соединить все части вместе.    
Итак, для поисковика нам понадобятся:         
**1. База документов **
> в первом дз - корпус Друзей    
в сегодняшнем дз - корпус юридических вопросов-ответов    
в итоговом проекте - корпус Авито   

**2. Функция индексации**                 
Что делает: собирает информацию о корпусе, по которуму будет происходить поиск      
Своя для каждого поискового метода:       
> A. для обратного индекса она создает обратный индекс (чудо) и сохраняет статистики корпуса, необходимые для Okapi BM25 (средняя длина документа в коллекции, количество доков ... )             
> B. для поиска через word2vec эта функция создает вектор для каждого документа в коллекции путем, например, усреднения всех векторов коллекции       
> C. для поиска через doc2vec эта функция создает вектор для каждого документа               

   Не забывайте сохранить все, что насчитает эта функция. Если это будет происходить налету во время поиска, понятно, что он будет работать сто лет     
   
**3. Функция поиска**     
Можно разделить на две части:
1. функция вычисления близости между запросом и документом    
> 1. для индекса это Okapi BM25
> 2. для w2v и d2v это обычная косинусная близость между векторами          
2. ранжирование (или просто сортировка)


Время все это реализовать.

In [466]:
import os
import json
from tqdm import tqdm_notebook
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from pymystem3 import Mystem
import pickle

In [467]:
import warnings
warnings.filterwarnings('ignore')

# Индексация
## Word2Vec
### Задание 1
Загрузите любую понравившуюся вам word2vec модель

In [468]:
from gensim.models import Word2Vec

In [469]:
model_path = 'araneum_none_fasttextcbow_300_5_2018.model'
w2v_model = Word2Vec.load(model_path)

### Задание 2 
Напишите функцию индексации для поиска через word2vec. Она должна для каждого документа из корпуса строить вектор.   
Все вектора надо сохранить, по формату советую json. При сохранении не забывайте, что вам надо сохранить не только  вектор, но и опознователь текста, которому он принадлежит. 
Для поисковика это может быть url страницы, для поиска по текстовому корпусу сам текст.

> В качестве документа для word2vec берите **параграфы** исходного текста, а не весь текст целиком. Так вектора будут более осмысленными. В противном случае можно получить один очень общий вектор, релевантый совершенно разным запросам.

In [470]:
from judicial_splitter import splitter

In [471]:
articles = os.listdir('article')[:1000]
articles = ['article' + os.sep + article for article in articles]

In [472]:
mystem = Mystem()

In [473]:
def preprocessing(input_text, del_stopwords=True, del_digit=True):
    """
    :input: raw text
        1. lowercase, del punctuation, tokenize
        2. normal form
        3. del stopwords
        4. del digits
    :return: lemmas
    """
    russian_stopwords = set(stopwords.words('russian'))
    words = [x.lower().strip(string.punctuation + '»«–…') for x in word_tokenize(input_text)]
    lemmas = [mystem.lemmatize(x)[0] for x in words if x]

    lemmas_arr = []
    for lemma in lemmas:
        if del_stopwords:
            if lemma in russian_stopwords:
                continue
        if del_digit:
            if lemma.isdigit():
                continue
        lemmas_arr.append(lemma)
        
    return lemmas_arr

In [474]:
def write_data(filename, data):
    with open(filename, 'w') as fout:
        json.dump(data, fout)

In [475]:
# def split_w2v(articles):
#     article_text = {}
#     w2v_paragraphs = []
    
#     for article in tqdm_notebook(articles):
#         with open(article, 'r', encoding='utf-8') as f:
#             f = f.read()
#             article_text[article] = f
#             splitted_text = splitter(f, 1)
# #             print(splitted_text)
            
#             for text in splitted_text:
#                 lemmas = preprocessing(text)
#                 w2v_paragraphs.append({'article_text': f, 'article_lemmas': lemmas})
                
#     return article_text, w2v_paragraphs

In [476]:
# w2v_a, w2v_p = split_w2v(articles)

In [477]:
def get_w2v_vectors(model, input_data):
    """Получает вектор документа"""
    vectors = []
    
    for input_d in input_data:
        try:
            vector = model.wv[input_d]
            vectors.append(vector)
        except KeyError as e:
#             print('KeyError - reason %s' % str(e))
            continue
    mean = sum(vectors) / len(vectors)        
    
    return mean

In [478]:
# def save_w2v_base(w2v_data):
#     """Индексирует всю базу для поиска через word2vec"""
#     w2v_result = []
#     article_vector = {}
    
#     for dictionary in tqdm_notebook(w2v_data):
#         w2v_vectors = get_w2v_vectors(w2v_model, dictionary['article_lemmas'])
#         article_vector = {'article_text': dictionary['article_text'], 'w2v_vectors': w2v_vectors.tolist()}
#         w2v_result.append(article_vector)
    
#     return w2v_result

In [479]:
# w2v_res = save_w2v_base(w2v_p)

In [480]:
def save_w2v_base(articles):
    """Индексирует всю базу для поиска через word2vec"""
    w2v_result = []
    article_vector = {}
    
    for article in tqdm_notebook(articles):
        with open(article, 'r', encoding='utf-8') as f:
            f = f.read()
            lemmas = preprocessing(f)
            w2v_vectors = get_w2v_vectors(w2v_model, lemmas)
            
            article_vector = {'article_text': f, 'w2v_vectors': w2v_vectors.tolist()}
            w2v_result.append(article_vector)
    
    return w2v_result

In [481]:
w2v_res = save_w2v_base(articles)

A Jupyter Widget

In [482]:
write_data('Word2Vec', w2v_res)

In [526]:
name_text = {}
for article in tqdm_notebook(articles):
    with open(article, 'r', encoding='utf-8') as fr:
        fr = fr.read()
        name_text[article] = fr

A Jupyter Widget

## Doc2Vec
### Задание 3
Напишите функцию обучения doc2vec на юридических текстах, и получите свою кастомную d2v модель. 
> Совет: есть мнение, что для обучения doc2vec модели не нужно удалять стоп-слова из корпуса. Они являются важными семантическими элементами.      

Важно! В качестве документа для doc2vec берите **параграфы** исходного текста, а не весь текст целиком. И не забывайте про предобработку.

In [484]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [485]:
def split_d2v(articles):
    article_text = {}
    d2v_paragraphs = []
    
    for article in tqdm_notebook(articles):
        with open(article, 'r', encoding='utf-8') as f:
            f = f.read()
            article_text[article] = f
            splitted_text = splitter(f, 4)
            
            for text in splitted_text:
                lemmas = preprocessing(text, del_stopwords=False)
                d2v_paragraphs.append({'article_text': f, 'article_lemmas': lemmas})
    
    return article_text, d2v_paragraphs

In [486]:
d2v_a, d2v_p = split_d2v(articles)

A Jupyter Widget

In [487]:
def train_doc2vec(input_data):
    d2v_data = [TaggedDocument(words=j['article_lemmas'], tags=[str(i)])for i, j in enumerate(input_data)]

    model = Doc2Vec(vector_size=300, alpha=0.025, min_alpha=0.025, min_count=0, workers=4, epochs=100)
    model.build_vocab(d2v_data)
    model.train(d2v_data, total_examples=model.corpus_count, epochs=model.epochs)
    
    return model

In [488]:
d2v_model = train_doc2vec(d2v_p)

### Задание 4
Напишите функцию индексации для поиска через doc2vec. Она должна для каждого документа из корпуса получать вектор.    
Все вектора надо сохранить, по формату советую json. При сохранении не забывайте, что вам надо сохранить не только вектор, но и опознователь текста, которому он принадлежит. 

In [489]:
def get_d2v_vectors(model, input_data):
    """Получает вектор документа"""
    d2v_vectors = model.infer_vector(input_data)
    
    return d2v_vectors

In [490]:
def save_d2v_base(d2v_data):
    """Индексирует всю базу для поиска через word2vec"""
    d2v_result = []
    article_vector = {}
    
    for dictionary in tqdm_notebook(d2v_data):
        d2v_vectors = get_d2v_vectors(d2v_model, dictionary['article_lemmas'])
        article_vector = {'article_text': dictionary['article_text'], 'd2v_vectors': d2v_vectors.tolist()}
        d2v_result.append(article_vector)
    
    return d2v_result

In [491]:
d2v_res = save_d2v_base(d2v_p)

A Jupyter Widget

In [492]:
write_data('Doc2Vec', d2v_res)

# Функция поиска

Для обратного индекса функцией поиска является Okapi BM25. Она у вас уже должна быть реализована.

Функция измерения близости между векторами нам пригодится:

In [493]:
from gensim import matutils
import numpy as np 

def similarity(v1, v2):
    v1_norm = matutils.unitvec(np.array(v1))
    v2_norm = matutils.unitvec(np.array(v2))
    return np.dot(v1_norm, v2_norm)

### Задание 5
Напишите функцию для поиска через word2vec и для поиска через doc2vec, которая по входящему запросу выдает отсортированную выдачу документов.

In [494]:
def search_w2v(query, model, w2v_res, n_results):
    result = {}
    final_results = []
    get_vectors = get_w2v_vectors(w2v_model, query)
    for w2v_r in w2v_res:
        compare_similarity = similarity(get_vectors, w2v_r['w2v_vectors'])
        result[compare_similarity] = w2v_r['article_text']
        
    for res in sorted(result, reverse=True)[:n_results]:
        final_results.append(result[res])
        
    return final_results

In [495]:
def search_d2v(query, model, d2v_res, n_results):
    result = {}
    final_results = []
    get_vectors = get_d2v_vectors(d2v_model, query)
    for d2v_r in d2v_res:
        compare_similarity = similarity(get_vectors, d2v_r['d2v_vectors'])
        result[compare_similarity] = d2v_r['article_text']
        
    for res in sorted(result, reverse=True)[:n_results]:
        final_results.append(result[res])
        
    return final_results

### Обратный индекс

In [496]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

In [497]:
def prepare_data(articles):
    inv_idx_result = []
    lengths = {}

    for article in tqdm_notebook(articles):
        with open(article, 'r', encoding='utf-8') as f:
            f = f.read()

        article_text = preprocessing(f)
        lengths[article] = len(article_text)
        inv_idx_result.append(' '.join(article_text))

    return inv_idx_result, lengths

In [498]:
inv_idx_res, l = prepare_data(articles)

A Jupyter Widget

In [499]:
avgdl = sum(l.values()) / len(l)

In [500]:
count_vect = CountVectorizer()
X = count_vect.fit_transform(inv_idx_res)
term_doc_matrix = pd.DataFrame(X.toarray(), index=articles, columns=count_vect.get_feature_names())

In [501]:
term_doc_matrix.head()

,00,000,01,02,024юл,03,04,05,06,07,...,ясно,ясногорск,ясность,ясный,ячейка,ячея,яшин,ященко,ящик,яя
article/220648.txt,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
article/186989.txt,0,0,0,3,0,1,0,1,0,4,...,0,0,0,0,0,0,0,0,0,0
article/47145.txt,0,0,0,0,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
article/132592.txt,0,0,1,0,0,0,3,0,3,0,...,0,0,0,0,0,0,0,0,0,0
article/27869.txt,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [502]:
def get_inv_idx(term_doc_matrix) -> dict:
    """
    Create inverted index by input doc collection
    :return: inverted index
    """
    count_idf = {}
    articles = term_doc_matrix.index.tolist()
    words = term_doc_matrix.columns.tolist()
    freq = term_doc_matrix.values.tolist()
    N = len(articles)

    for i, j in enumerate(words):
        n = 0
        for f in freq:
            count = f[i]
            if count != 0:
                n += 1

        idf = log((N - n + 0.5) / (n + 0.5))
        count_idf[j] = idf
    
    return count_idf

In [503]:
c_idf = get_inv_idx(term_doc_matrix)

In [504]:
from math import log

k1 = 2.0
b = 0.75

def score_BM25(idf, qf, dl, avgdl, k1, b) -> float:
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
    return idf * (k1 + 1) * qf / (qf + k1 * (1 - b + b * dl / avgdl))

In [505]:
def get_search_result(query, n_results) -> list:
    """
    Compute sim score between search query and all documents in collection
    Collect as pair (doc_id, score)
    :param query: input text
    :return: list of lists with (doc_id, score)
    """
    inv_idx_result = {article: 0 for article in term_doc_matrix.index.tolist()}

    for q in query:
        if q in term_doc_matrix.columns.tolist():
            idx = term_doc_matrix.columns.tolist().index(q)
            for i, article in enumerate(term_doc_matrix.index.tolist()):
                qf = term_doc_matrix.values.tolist()[i][idx]
                dl = l[article]
                idf = c_idf[q] 
                okapi = score_BM25(idf, qf, dl, avgdl, k1, b)
                inv_idx_result[article] += okapi
            
    return sorted(inv_idx_result.items(), key=lambda k: k[1], reverse=True)[:n_results]

После выполнения всех этих заданий ваш поисковик готов, поздравляю!                  
Осталось завернуть все написанное в питон скрипт, и сделать общую функцию поиска гибким, чтобы мы могли искать как по обратному индексу, так и по word2vec, так и по doc2vec.          
Сделать это можно очень просто через старый добрый ``` if ```, который будет дергать ту или иную функцию поиска:

In [533]:
def search(query, search_method, n_results=5):
    if search_method == 'inverted_index':
        final = []
        query = preprocessing(query)
        res_ii = get_search_result(query, n_results)
        for f in res_ii:
            f = f[0]
            final.append(name_text.get(f))
            
    elif search_method == 'word2vec':
        query = preprocessing(query)
        final = search_w2v(query, w2v_model, w2v_res, n_results)
        
    elif search_method == 'doc2vec':
        query = preprocessing(query, del_stopwords=False)
        final = search_d2v(query, d2v_model, d2v_res, n_results)
    
    else:
        raise TypeError('unsupported search method')
        
    return final

In [534]:
search('база', 'inverted_index')

['\n\nКоллегия судей Высшего Арбитражного Суда Российской Федерации в составе председательствующего судьи Мурина О.Л., судей Зориной М.Г. и Поповченко А.А., рассмотрев в судебном заседании заявление муниципального учреждения "Служба заказчика жилищно-коммунальных услуг" (ул.  Смышляева , 25, г. Лысьва, Пермский край, 618900) о пересмотре в порядке надзора постановления Федерального арбитражного суда Уральского округа от 29.05.2007 по делу N А50-17562/2006-А12 Арбитражного  суда Пермской области, установила: муниципальное учреждение "Служба заказчика жилищно-коммунальных услуг" обратилось в арбитражный суд с заявлением о признании недействительным решения от 14.07.2006 N 493 Межрайонной инспекции ФНС России N 6 по Пермскому краю (пр.   Победы, 34, г. Лысьва, Пермский край, 618900) об отказе в привлечении налогоплательщика к ответственности за совершение налогового правонарушения (с учетом уточнения предмета требования в порядке статьи 49 Арбитражного процессуального кодекса Российской Ф

In [449]:
search('владеть', 'word2vec')

['ВЕРХОВНЫЙ СУД РОССИЙСКОЙ ФЕДЕРАЦИИ\nКАССАЦИОННОЕ ОПРЕДЕЛЕНИЕ\nот 22 августа 2006 года\n\nДело N 77-о06-6 Судебная коллегия по уголовным делам Верховного Суда Российской Федерации в составе: \xa0\xa0\xa0  \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0  Галиуллина \xa0\xa0\xa0  \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0  \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0  рассмотрела в судебном заседании кассационные жалобы осужденных М., Т. и законного представителя несовершеннолетнего Т. - М.Е. на приговор Липецкого областного суда от 24 апреля 2006 года, которым М., <...>, судимый: 09.08.2005 года по ст. 158 ч. 2 п. п. "а", "б", "в" УК РФ к 2 годам исправ

In [450]:
search('владеть', 'doc2vec')

['ВЕРХОВНЫЙ СУД РОССИЙСКОЙ ФЕДЕРАЦИИ\nОПРЕДЕЛЕНИЕ\nот 14 апреля 2003 г. N 15-Д03-3\n\nПредседательствующий: Тарасов А.В. Судебная коллегия по уголовным делам Верховного Суда Российской Федерации в составе: председательствующего Свиридова Ю.А. судей Яковлева В.К.,  Колышкина  В.И. рассмотрела в судебном заседании от 14 апреля 2003 года надзорную жалобу осужденного К. на приговор Верховного Суда Республики Мордовия от 7 сентября 2001 года, которым К., <...>, ранее не судимый, - осужден по ст. 105 ч. 2 п. "в" УК РФ к лишению свободы на 12 лет в исправительной колонии строгого режима. Ш., <...>, ранее не судимая, - осуждена по ст. 33 ч. ч. 4, 5, 105 ч. 2 п. "в" УК РФ, с применением ст. 64 УК РФ, на 5 лет лишения свободы условно, с испытательным сроком на 5 лет на основании ст. 73 УК РФ. В кассационном порядке дело не рассматривалось. В отношении Ш. дело рассматривается в порядке ст. 410 УПК РФ. Заслушав доклад судьи Яковлева В.К., мнение прокурора Архиповой Л.И., поддержавшей доводы надзор